In [76]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CS\ 224N/CS\ 224N\ Project
! ls # verify that you are in the right directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/CS 224N/CS 224N Project'
/content/drive/MyDrive/CS 224N/CS 224N Project
'224N Project Brainstorm.gdoc'	        drive
'224N Project Helpful Tutorials.gdoc'  'first proposal OLD'
'224N Project Milestone Notes.gdoc'     gpt2-baseline.ipynb
 aita_clean.csv			        logs
 aita_comments.csv		       'Reddit Scraper.ipynb'
 aita_test_set.csv		        results
 aita_train_set.csv		        T5_Attempt_2.ipynb
 aita_valid_set.csv		        T5-baseline.ipynb
 bart-baseline-attempt2.ipynb	        t5_checkpoints
 bert-baseline.ipynb		        train.pt
 csvs				        wandb
 dataset_agg.ipynb


In [77]:
import os
import pandas as pd
path = os.listdir('csvs')
print(path)

['comments_batch_8273.csv', 'comments_batch_11473.csv', 'comments_batch_4067.csv', 'comments_batch_15737.csv', 'comments_batch_19402.csv', 'comments_batch_21869.csv', 'comments_batch_60700.csv', 'comments_batch_62058.csv', 'comments_batch_24666.csv', 'comments_batch_62270.csv', 'comments_batch_63000.csv', 'comments_batch_25587.csv', 'comments_batch_63037.csv', 'comments_batch_64000.csv', 'comments_batch_65000.csv', 'comments_batch_26957.csv', 'comments_batch_33231.csv', 'comments_batch_65660.csv', 'comments_batch_66000.csv', 'comments_batch_27485.csv', 'comments_batch_66593.csv', 'comments_batch_33688.csv', 'comments_batch_67000.csv', 'comments_batch_68000.csv', 'comments_batch_69000.csv', 'comments_batch_69361.csv', 'comments_batch_70000.csv', 'comments_batch_37064.csv', 'comments_batch_71000.csv', 'comments_batch_72000.csv', 'comments_batch_73000.csv', 'comments_batch_74000.csv', 'comments_batch_75000.csv', 'comments_batch_76000.csv', 'comments_batch_77000.csv', 'comments_batch_78000

In [ ]:
df = pd.DataFrame()
# %cd csvs
# ! ls
for i, f in enumerate(path):
  df_new = pd.read_csv(f)
  # df_new = pd.read_csv(os.path.join(os.getcwd(), f))

  df = pd.concat([df, df_new])

# remove duplicate ids
df = df.drop_duplicates(subset=['id'])
print(df.head())

/content/drive/MyDrive/CS 224N/CS 224N Project/csvs
comments_batch_11473.csv  comments_batch_63037.csv  comments_batch_80000.csv
comments_batch_15737.csv  comments_batch_64000.csv  comments_batch_81000.csv
comments_batch_19402.csv  comments_batch_65000.csv  comments_batch_82000.csv
comments_batch_21869.csv  comments_batch_65660.csv  comments_batch_8273.csv
comments_batch_24666.csv  comments_batch_66000.csv  comments_batch_83000.csv
comments_batch_25587.csv  comments_batch_66593.csv  comments_batch_84000.csv
comments_batch_26957.csv  comments_batch_67000.csv  comments_batch_85000.csv
comments_batch_27485.csv  comments_batch_68000.csv  comments_batch_86000.csv
comments_batch_32000.csv  comments_batch_69000.csv  comments_batch_87000.csv
comments_batch_33231.csv  comments_batch_69361.csv  comments_batch_88000.csv
comments_batch_33688.csv  comments_batch_70000.csv  comments_batch_89000.csv
comments_batch_37064.csv  comments_batch_71000.csv  comments_batch_90000.csv
comments_batch_4067.csv  

In [ ]:
print(len(df))

87215


In [75]:
# Remove any elements that are [deleted] from the dataframe
# also remove [removed] posts --> these are posts the mods deleted
df = df[~df['comments'].str.contains('\[deleted\]')]
print(len(df))
df = df[~df['comments'].str.contains('\[removed\]')]
print(len(df))

df = df[~df['comments'].str.contains('\[contact the moderator')]


83929
83929
83610


In [ ]:
# %cd ..
df_clean = pd.read_csv("aita_clean.csv")
print(len(df_clean))
df_combined = pd.merge(df, df_clean, on='id', how='inner')
print(len(df_combined))

/content/drive/MyDrive/CS 224N/CS 224N Project
97628
83929


In [ ]:
# save the new data frame in a file called "aita_comments.csv"
df_combined.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/aita_comments.csv", index=False) 

Conduct final preprocessing on our dataset to structure it appropriately for our generative model

In [ ]:
import os
import pandas as pd

In [92]:
df = pd.read_csv("aita_comments.csv")
df = df[~df['comments'].str.contains('\[deleted\]')]
print(len(df))
df = df[~df['comments'].str.contains('\[removed\]')]
print(len(df))

df = df[~df['comments'].str.contains('\[contact the moderator')]

83929
83929


In [93]:
print(len(df))

83610


In [94]:
df["text"] = df["title"].astype(str) +". "+ df["body"].astype(str)

# create a one hot vector for the verdicts
# ["asshole", "everyone sucks", "no assholes here", "not the asshole"]
# this gets changed to ["v_yta", "v_esh", "v_nah", "v_nta"] --> v stands for verdict and the verdict after is abbreviated
one_hot = pd.get_dummies(df['verdict'])
one_hot = one_hot.rename(columns = {'asshole': 'v_yta', 'everyone sucks': 'v_esh', 'no assholes here': 'v_nah', 'not the asshole': 'v_nta'})

df = pd.concat([df, one_hot], axis=1)

df = df[["text", "comments", "is_asshole", "v_yta", "v_esh", "v_nah", "v_nta"]] # drop unused columns 


In [81]:
# normalize text length based on average and mean length of posts + title combo
avg_words = df['comments'].str.split().str.len().mean()
avg_words_text = df['text'].str.split().str.len().mean()

print(f'Average comment length is: ${avg_words}')
print(f'Average text length is: ${avg_words_text}')

median_words = df['comments'].str.split().str.len().median()
median_words_text = df['text'].str.split().str.len().median()
print(f'Median comment length is: ${median_words}')
print(f'Median text length is: ${median_words_text}')


Average comment length is: $49.08097117569669
Average text length is: $348.58258581509386
Median comment length is: $37.0
Median text length is: $320.0


In [82]:
cutoff = 350
num_long_comments = (df['text'].apply(lambda x: len(x.split(' '))) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))
# if we were to set our cap at 400, these are the number of comments that'd be 
# affected. We also display the percent of the total that will get truncated
cutoff = 400
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

cutoff = 450
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

cutoff = 500
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

cutoff = 550
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

cutoff = 600
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

The number of comments that are truncated with cutoff: 350 and its percentage of the total dataset
36195
0.43290276282741297
The number of comments that are truncated with cutoff: 400 and its percentage of the total dataset
28934
0.34605908384164574
The number of comments that are truncated with cutoff: 450 and its percentage of the total dataset
22404
0.2679583781844277
The number of comments that are truncated with cutoff: 500 and its percentage of the total dataset
16956
0.20279870828848223
The number of comments that are truncated with cutoff: 550 and its percentage of the total dataset
11788
0.14098792010525057
The number of comments that are truncated with cutoff: 600 and its percentage of the total dataset
4949
0.059191484272216245


We chose 508 as the cutoff because the T5 baseline can support at most 512 words. We wanted to include the last 4 words as the prompt "Am I the Asshole" because our research found that including a prompt at the end of the text sample greatly helped generative models generate text. 

508 is also a healthy cutoff because less than 1/5 of our training data will be truncated, it's 19.335% to be exact. 


In [95]:
print(len(df))

83610


In [84]:
cutoff = 508
num_long_comments = (df['text'].apply(lambda x: len(x.split())) > cutoff).sum()
print(f'The number of comments that are truncated with cutoff: {cutoff} and its percentage of the total dataset')
print(num_long_comments)
print(num_long_comments / len(df))

# Truncate all the strings in the dataset longer than cutoff 
df['text'] = df['text'].apply(lambda x: ' '.join(x.split(maxsplit=cutoff)[:cutoff]))



# add a prompt to help generative model with prompt engineering 
question = ". Am I the asshole?"
df["text"] = df["text"].astype(str) + question

avg_words_text_final = df['text'].str.split().str.len().mean()
print(f'Average text length is: ${avg_words_text_final}')

median_words_text_final = df['text'].str.split().str.len().median()
print(f'Median text length is: ${median_words_text_final}')


The number of comments that are truncated with cutoff: 508 and its percentage of the total dataset
16152
0.1931826336562612
Average text length is: $330.57955986126063
Median text length is: $324.0


In [85]:
print(len(df))


83610


In [96]:
# Print counts of classes
# print(f"Count of the YTA label: {}")
size = len(df)
ratios = {}
ratios['v_yta'] = df['v_yta'].value_counts()[1] / len(df)
ratios['v_esh'] = df['v_esh'].value_counts()[1] / len(df)
ratios['v_nah'] = df['v_nah'].value_counts()[1] / len(df)
ratios['v_nta'] = df['v_nta'].value_counts()[1] / len(df)

ratios

{'v_yta': 0.2129051548857792,
 'v_esh': 0.05647649802655185,
 'v_nah': 0.12212653988757326,
 'v_nta': 0.6084918072000957}

In [97]:
# Sample the test and valid dataset to fit the above distribution
test_valid_size = 1000
yta_df = df.loc[df['v_yta'] == 1] 
esh_df = df.loc[df['v_esh'] == 1] 
nah_df = df.loc[df['v_nah'] == 1] 
nta_df = df.loc[df['v_nta'] == 1] 

valid_df_yta = yta_df.sample(n = int(ratios["v_yta"] * 1000))
valid_df_esh = esh_df.sample(n = int(ratios["v_esh"] * 1000))
valid_df_nah = nah_df.sample(n = int(ratios["v_nah"] * 1000))
valid_df_nta = nta_df.sample(n = int(ratios["v_nta"] * 1000))
valid_df = pd.concat([valid_df_yta, valid_df_esh, valid_df_nah, valid_df_nta])
df = df.loc[~df.index.isin(valid_df.index)]

valid_set = valid_df.reset_index(drop=True)
df = df.reset_index(drop=True)
print(len(df))

print(len(valid_set))

yta_df = df.loc[df['v_yta'] == 1] 
esh_df = df.loc[df['v_esh'] == 1] 
nah_df = df.loc[df['v_nah'] == 1] 
nta_df = df.loc[df['v_nta'] == 1] 

test_df_yta = yta_df.sample(n = int(ratios["v_yta"] * 1000))
test_df_esh = esh_df.sample(n = int(ratios["v_esh"] * 1000))
test_df_nah = nah_df.sample(n = int(ratios["v_nah"] * 1000))
test_df_nta = nta_df.sample(n = int(ratios["v_nta"] * 1000))
test_df = pd.concat([test_df_yta, test_df_esh, test_df_nah, test_df_nta])
df = df.loc[~df.index.isin(test_df.index)]

test_set = test_df.reset_index(drop=True)
df = df.reset_index(drop=True)
print(len(df))
print(len(test_set))


82612
998
81614
998


In [24]:
# #Split validation and test set
# valid_set = df.sample(n = 1000)
# df = df.loc[~df.index.isin(valid_set.index)]

# #Reset the indexes
# valid_set = valid_set.reset_index()
# df = df.reset_index()

# test_set = df.sample(n = 1000)
# df = df.loc[~df.index.isin(test_set.index)]

# #Reset the indexes
# test_set = test_set.reset_index()
# df = df.reset_index()

In [98]:
# save the new data frames "
df.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/aita_train_set.csv", index=False) 
valid_set.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/aita_valid_set.csv", index=False)
test_set.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/aita_test_set.csv", index=False)